In [83]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [40]:
!git config --global user.email dpsdanilpar@gmail.com
!git config --global user.name Unknown030405
!git pull origin main

Merge made by the 'recursive' strategy.
 Conformer. Audio Preprocessing.ipynb | 51145 +++++++++++++++++++++++++++++++++
 src/augmentation.py                  |   137 +
 2 files changed, 51282 insertions(+)
 create mode 100644 Conformer. Audio Preprocessing.ipynb
 create mode 100644 src/augmentation.py


From https://github.com/nakhodnov17/conformer
 * branch            main       -> FETCH_HEAD


In [84]:
from IPython.display import display, HTML, Video
display(HTML("<style>.container { width:90% !important; }</style>"))

In [50]:
%pip install matplotlib_inline

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


# Load data

In [85]:
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader

import sentencepiece

In [86]:
import os
import glob
import json
import regex

import tqdm.notebook as tqdm

import numpy as np
import pandas as pd

from ipywidgets import GridBox, Audio, HBox, VBox, Box, Label, Layout

import matplotlib.pyplot as plt
# import matplotlib_inline

# %matplotlib inline
# matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [87]:
base_path = '/home/jupyter/mnt/datasets'

libri_speech_base_path = os.path.join(base_path, 'LibriSpeech_ds')
golos_base_path = os.path.join(base_path, 'golos')

In [88]:
from src.dataset import get_libri_speech_dataset, get_golos_dataset

In [89]:
def sample_dataset(dataset, n=4):
    grid = []
    for idx in range(n):
        grid.append(
            VBox([
                Label('{0:d}, {1}, {2:.1f}'.format(idx, dataset['text'][idx], dataset['duration'][idx])),
                Audio.from_file(dataset['audio_path'][idx], autoplay=False, loop=False),
            ])
        )
    
    return HBox([VBox(grid[0::2]), VBox(grid[1::2])])

In [90]:
libri_speech_dev = get_libri_speech_dataset(libri_speech_base_path, split='dev')
libri_speech_test = get_libri_speech_dataset(libri_speech_base_path, split='test')
libri_speech_train = get_libri_speech_dataset(libri_speech_base_path, split='train')

print('Loaded {0:d} objects'.format(len(libri_speech_dev['audio_path'])))
print('Loaded {0:d} objects'.format(len(libri_speech_test['audio_path'])))
print('Loaded {0:d} objects'.format(len(libri_speech_train['audio_path'])))

Loaded 1400 objects
Loaded 1352 objects
Loaded 54472 objects


In [ ]:
sample_dataset(libri_speech_dev)

In [ ]:
sample_dataset(libri_speech_test)

In [ ]:
sample_dataset(libri_speech_train)

In [ ]:
golos_train = get_golos_dataset(golos_base_path, split='train')
golos_test_crowd = get_golos_dataset(golos_base_path, split='test/crowd')
golos_test_farfield = get_golos_dataset(golos_base_path, split='test/farfield')

print('Loaded {0:d} objects'.format(len(golos_train['audio_path'])))
print('Loaded {0:d} objects'.format(len(golos_test_crowd['audio_path'])))
print('Loaded {0:d} objects'.format(len(golos_test_farfield['audio_path'])))

In [ ]:
sample_dataset(golos_train)

In [ ]:
sample_dataset(golos_test_crowd)

In [ ]:
sample_dataset(golos_test_farfield)

In [91]:
train_datasets = {
    'libri_speech/train': libri_speech_train, 
#     'golos/train': golos_train
}

test_datasets = {
#     'golos/test/crowd': golos_test_crowd,
#     'golos/test/farfield': golos_test_farfield,
    'libri_speech/dev': libri_speech_dev,
    'libri_speech/test': libri_speech_test,
}

In [70]:
libri_speech_train

,audio_path,text,duration
0,/home/jupyter/mnt/datasets/LibriSpeech_ds/trai...,он сделал это так неловко что задел образок мо...,9.76
1,/home/jupyter/mnt/datasets/LibriSpeech_ds/trai...,я высунул нос из под одеяла остановил рукою об...,11.30
2,/home/jupyter/mnt/datasets/LibriSpeech_ds/trai...,он же в пестром ваточном халате подпоясанном п...,16.08
3,/home/jupyter/mnt/datasets/LibriSpeech_ds/trai...,отчего он не бьет мух около володи ной постели...,4.01
4,/home/jupyter/mnt/datasets/LibriSpeech_ds/trai...,нет володя старше меня а я меньше всех оттого ...,4.04
...,...,...,...
54467,/home/jupyter/mnt/datasets/LibriSpeech_ds/trai...,все персоны захлопали и лорд епископ и городск...,6.46
54468,/home/jupyter/mnt/datasets/LibriSpeech_ds/trai...,я уже не хочу описывать как мне показывали зав...,5.20
54469,/home/jupyter/mnt/datasets/LibriSpeech_ds/trai...,достаточно будет одной детали,2.20
54470,/home/jupyter/mnt/datasets/LibriSpeech_ds/trai...,впереди шел епископ он председательствовал на ...,15.10


# Create tokenizer

In [92]:
# Save text from all train datasets to file
### YOUR CODE HERE
with open("texts.txt", "w") as file:
    for value in train_datasets.values():
        for elm in value["text"]:
#             print(elm)
            file.write(elm + '\n')

In [93]:
# Train sentencepiece tokenizer
sentencepiece.SentencePieceTrainer.train(
    input='texts.txt', model_prefix='tokenizer', vocab_size=128, model_type='unigram',
    bos_id=-1, eos_id=-1, character_coverage=1.0
)

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: texts.txt
  input_format: 
  model_prefix: tokenizer
  model_type: UNIGRAM
  vocab_size: 128
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: -1
  eos_id: -1
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_privacy_noise_level: 0
  differential_privacy_clipping_th

In [94]:
# Load tokenizer model
sp_tokenizer = sentencepiece.SentencePieceProcessor(model_file='tokenizer.model')

In [95]:
# Check if it is working
sp_tokenizer.encode_as_ids(['привет, как тебя зовут'])

[[89, 8, 62, 0, 87, 1, 47, 25, 11, 1, 17, 15, 8, 12, 9]]

In [96]:
print(sp_tokenizer.encode_as_pieces(['привет, как тебя зовут']))

[['▁при', 'в', 'ет', ',', '▁как', '▁', 'те', 'б', 'я', '▁', 'з', 'о', 'в', 'у', 'т']]


# Create Dataset

In [97]:
from src.dataset import AudioDataset, collate_fn

In [98]:
ds_train = AudioDataset(pd.concat(train_datasets.values()), sp_tokenizer, min_duration=1.36, max_duration=10.96)

ds_test_dict = {
    name: AudioDataset(dataset, sp_tokenizer, min_duration=1.36, max_duration=10.96)
    for name, dataset in test_datasets.items()
}
ds_train_dict = {
    name: AudioDataset(dataset, sp_tokenizer, min_duration=1.36, max_duration=10.96)
    for name, dataset in train_datasets.items()
}

/usr/local/lib/python3.8/dist-packages/pandas/core/computation/expressions.py:190: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pandas/core/computation/expressions.py:190: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pandas/core/computation/expressions.py:190: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pandas/core/computation/expressions.py:190: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(


In [99]:
ds_train[0]

('/home/jupyter/mnt/datasets/LibriSpeech_ds/train/audio/8086/11365/shortstories_11_garshin_0260.wav',
 tensor([-3.0518e-05,  0.0000e+00,  9.1553e-05,  ..., -1.6235e-02,
         -2.4017e-02, -3.4882e-02]),
 21760,
 'быть может еще успею',
 tensor([ 75,  56,  90,  61,   9,   1,   2, 126,   2,  51,   7,  34,   2,  28]),
 14)

# Create Dataloader

In [100]:
batch_size = 8
num_workers = 0

dl_train = DataLoader(
    ds_train, batch_size=batch_size, shuffle=True,
    num_workers=num_workers, pin_memory=False, collate_fn=collate_fn
)

dl_test_dict = {
    name: DataLoader(
        dataset, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=False, collate_fn=collate_fn
    )
    for name, dataset in ds_test_dict.items()
}
dl_train_dict = {
    name: DataLoader(
        dataset, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=False, collate_fn=collate_fn
    )
    for name, dataset in ds_train_dict.items()
}

In [ ]:
next(iter(dl_train))